In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import RV_dynmc.RV_dynmc as rvd
import astropy.constants as cst

Msun = cst.M_sun.value
Me = cst.M_earth.value

In [4]:
names = ['M0','M1','M2','P1','P2','e1','e2','w1','w2','W1','W2','phi1','phi2','inc1','inc2','vsys1_UVES','vsys2_UVES','jit1_UVES','jit2_UVES']
dists = ['Uniform','Uniform','LogUniform','Gaussian','LogUniform','Gaussian','Uniform','Gaussian','Gaussian','Gaussian','Uniform','Uniform','Uniform','Gaussian','Uniform','Gaussian','Gaussian','LogUniform','LogUniform']
As = [0.033,0.033,1*Me/Msun,  20.907,60,0.36,0   ,4.9,0   ,0   ,0      ,0      ,0      ,88.5*np.pi/180,-np.pi,-11830,-11880,0.1,0.1]
Bs = [0.005,0.005,10000*Me/Msun,0.005,10000,0.01,0.01,0.1,0.01,0.01,2*np.pi,2*np.pi,2*np.pi,0.1*np.pi/180 ,np.pi,100   ,100   ,40,40]

mus = [0.033,0.033,1000*Me/Msun  ,20.907,1000,0.36,0.001 ,4.9,0   ,0   ,3.1,5.58,3.1,88.5*np.pi/180,0,-11830,-11880,10,10]
sigmas = [0.001,0.001,300*Me/Msun,0.001,300.  ,0.01,0.0001,0.1,0.01,0.01,1.4,0.01,0.4,0.1*np.pi/180 ,1,10   ,10   ,1,1]

In [5]:
model = rvd.RV_dynmc(n_orbits=2,n_lines=2,insts=['UVES'],chains=50,steps=80000)

In [8]:
model.define_priors(names,dists,As,Bs)
data = pd.read_csv('2M1510_rvs.rdb',sep='\t',header=0,skiprows=[1])
model.add_rv_data(['UVES'],[data],['kms'])
model.x0_from_values(mus,sigmas)

In [ ]:
model.run() #may take a few days

In [ ]:
sam = model.sample_analysis

In [ ]:
sam.discard = 30000 #can edit these
sam.thin = 100 #can edit these

In [ ]:
samp = sam.random_subsample(600)
sims = sam.sims_subsample(samp)

In [ ]:
fig,ax = plt.subplots(figsize=[8,8],tight_layout=True)
ax.set(xlim=[-1,1],ylim=[-1,1])
Omegas = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
inclins = [0., 0.33069396,0.7, 0.99208189, 1.32277585, 1.65346982, 1.98416378, 2.31485774, 2.64555171, 2.97624567, 3.4376336, 3.7076336, 3.96832756, 4.29902153, 4.62971549,4.96040945, 5.29110342,5.5, 5.62179738, 5.75249134, 6.11318531]
inclins2 = [0., 0.33069396, 0.99208189, 1.32277585, 1.65346982, 1.98416378, 2.18485774, 2.64555171, 2.97624567, 3.86832756, 4.29902153, 4.62971549,4.96040945, 5.29110342, 5.52179738, 5.75249134, 6.11318531]
c2 = 'lightsalmon'
c1 = 'firebrick'
colours = [c1,c2,'mediumblue']
colours1 = [c1,c1,c2,c2,c2,c2,c2,c2,c1,c1,c1,c1,c2,c2,c2,c2,c2,c2,c1,c1,c1]
colours2 = [c2,c2,c1,c1,c1,c1,c2,c2,c2,c2,c1,c1,c1,c1,c2,c2,c2]
for j,inc2 in enumerate(inclins):
    M0 = 0.033
    Ms,Ps,es,ws,Ws,fs,incs = [0.033,0],[20,110],[0.36,0.01],[4.9,0],[0,Omegas[j]],[0,0],[1.55,inc2]
    sim = rebound.Simulation()
    sim.units = ('days', 'AU', 'Msun')

    sim.add(m=M0)
    for i,m in enumerate(Ms): 
        sim.add(m=m,P=Ps[i],e=es[i],omega=ws[i],Omega=Ws[i],M=fs[i],inc=incs[i])

    sim.move_to_com()

    icw = []
    isw = []
    icw2 = []
    isw2 = []
    times = np.arange(0,18000,10)
    for t in times:
        sim.integrate(t)
        ip = sim.particles[2].inc
        ib = sim.particles[1].inc
        Wp = sim.particles[2].Omega
        Wb = sim.particles[1].Omega
        wp = sim.particles[2].omega
        wb = sim.particles[1].omega

        x = sim.particles[2].x
        y = sim.particles[2].y
        z = sim.particles[2].z
        dx = sim.particles[2].vx
        dy = sim.particles[2].vy
        dz = sim.particles[2].vz

        Tinv = fcs.frame_trans_inv(Wb,ib,wb)
        r = np.array([x,y,z])
        v = np.array([dx,dy,dz])
        R = np.matmul(Tinv,r)
        V = np.matmul(Tinv,v)
        X,Y,Z = R[0],R[1],R[2]
        dX,dY,dZ = V[0],V[1],V[2]

        I = I_xyz(X,Y,Z,dX,dY,dZ)
        W = W_xyz(X,Y,Z,dX,dY,dZ)

        icw2.append((ip)*np.cos(Wp-Wb-wb)/np.pi)
        isw2.append((ip)*np.sin(Wp-Wb-wb)/np.pi)
        icw.append(I*np.cos(W)/np.pi)
        isw.append(I*np.sin(W)/np.pi)

    ax.plot(icw,isw,c=colours1[j],alpha=1)
    ax.set(xlabel=r'I$_{\rm p}~\cos(\Omega_{\rm p}$)',ylabel=r'I$_{\rm p}~\sin(\Omega_{\rm p}$)')

for i,sim in tqdm(enumerate(sims)):
    if True==True:
        icw = []
        isw = []
        times = np.arange(0,20000,100)
        for t in times:
            sim.integrate(t)
            ip = sim.particles[2].inc
            ib = sim.particles[1].inc
            Wp = sim.particles[2].Omega
            Wb = sim.particles[1].Omega
            wp = sim.particles[2].omega
            wb = sim.particles[1].omega
            inc = imut(ib,ip,Wb,Wp)

            x = sim.particles[2].x
            y = sim.particles[2].y
            z = sim.particles[2].z
            dx = sim.particles[2].vx
            dy = sim.particles[2].vy
            dz = sim.particles[2].vz
    
            Tinv = fcs.frame_trans_inv(Wb,ib,wb)
            r = np.array([x,y,z])
            v = np.array([dx,dy,dz])
            R = np.matmul(Tinv,r)
            V = np.matmul(Tinv,v)
            X,Y,Z = R[0],R[1],R[2]
            dX,dY,dZ = V[0],V[1],V[2]
    
            I = I_xyz(X,Y,Z,dX,dY,dZ)
            W = W_xyz(X,Y,Z,dX,dY,dZ)

            
            icw.append((I)*np.cos(W)/np.pi)
            isw.append((I)*np.sin(W)/np.pi)

        ax.plot(icw,isw,c=colours[2],alpha=0.05,zorder=5)  